
# KOR-END 번역 모델 만들기

이번 특강에서는 한국어 문장을 영어로 번역하는 시퀀스-투-시퀀스(sequence-to-sequence, seq2seq) 모델을 학습하는 방법을 알아보겠습니다.

필요 라이브러리: ``torchtext``, ``spacy`` 를 사용하여 데이터셋을 전처리(preprocess)합니다.

## Import

#### Download Requirements

In [ ]:
!pip install --upgrade git+https://github.com/dAiv-CNU/torchdaiv.git

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download ko_core_news_sm

#### Library Imports

In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import spacy
from torchdaiv import datasets
#from torchdaiv.lectures.kor_eng_translator import nn
from torchdaiv.lectures.kor_eng_translator.util import vocabulary, transforms

from rich.traceback import install
#install(show_locals=True)

%matplotlib inline

---
## Text Preprocess (with Spacy)

``torchtext`` 에는 언어 변환 모델을 만들 때 쉽게 사용할 수 있는 데이터셋을 만들기 적합한 다양한 도구가 있습니다.
이 예제에서는 가공되지 않은 텍스트 문장(raw text sentence)을 토큰화(tokenize)하고, 어휘집(vocabulary)을 만들고,
토큰을 텐서로 숫자화(numericalize)하는 방법을 알아보겠습니다.

| (다만, torchtext는 2024년 4월 이후 더 이상 업데이트가 진행되지 않는다는 점에 유의해야 합니다.)

아래를 실행하여 Spacy 토크나이저가 쓸 한국어와 영어에 대한 데이터를 다운로드 받습니다.

In [2]:
# spacy tokenizer 적용
ko_core_news_sm = spacy.load("ko_core_news_sm")
en_core_web_sm = spacy.load("en_core_web_sm")

In [3]:
ko_tokenizer = lambda x: [token.text for token in ko_core_news_sm(x)]
en_tokenizer = lambda x: [token.text for token in en_core_web_sm(x)]

In [4]:
from spacy.lang.ko.examples import sentences

# 작동 확인
doc = ko_core_news_sm(sentences[0])
print("Original:", doc.text)
print("Tokenized:", ko_tokenizer(sentences[0]), end="\n\n")

for token in doc:
    print(">", token.text, f"({token.lemma_}) |", token.pos_, token.dep_)

Original: 애플이 영국의 스타트업을 10억 달러에 인수하는 것을 알아보고 있다.
Tokenized: ['애플이', '영국의', '스타트업을', '10억', '달러에', '인수하는', '것을', '알아보고', '있다', '.']

> 애플이 (애플+이) | NOUN dislocated
> 영국의 (영국+의) | PROPN nmod
> 스타트업을 (스타트업+을) | NOUN nsubj
> 10억 (10+억) | NUM compound
> 달러에 (달러+에) | ADV obl
> 인수하는 (인수+하+는) | VERB acl
> 것을 (것+을) | NOUN obj
> 알아보고 (알아보+고) | AUX ROOT
> 있다 (있+다) | AUX aux
> . (.) | PUNCT punct


---
## Load Dataset
using spacy

In [5]:
# 데이터셋 로드 - 아무 처리도 하지 않았을 때
train_dataset = datasets.AnkiKorEngDataset("./data", split_rate=(0.5, 0.3, 0.2))
valid_dataset = datasets.AnkiKorEngDataset("./data", valid=True, split_rate=(0.5, 0.3, 0.2))
test_dataset = datasets.AnkiKorEngDataset("./data", test=True, split_rate=(0.5, 0.3, 0.2))

Extraction completed.
Dataset loaded. 2945 samples loaded.
Extraction completed.
Dataset loaded. 1767 samples loaded.
Extraction completed.
Dataset loaded. 1177 samples loaded.


In [6]:
# 데이터셋 형태 확인
sample = list(zip(*train_dataset[0:5]))+list(zip(*train_dataset[500:505]))
for i, (kor, eng) in enumerate(sample):
    print(i, kor, eng)

0 가. Go.
1 안녕. Hi.
2 뛰어! Run!
3 뛰어. Run.
4 누구? Who?
5 천천히 걸어. Walk slowly.
6 내가 틀렸나? Was I wrong?
7 우리는 아프다. We are sick.
8 우리는 계란을 먹었다. We ate eggs.
9 우린 약속했어. We promised.


#### Vocabulary 생성

In [7]:
ko_vocab = vocabulary.build_vocab(raw_dataset=train_dataset.raw_kor, tokenizer=ko_tokenizer)
en_vocab = vocabulary.build_vocab(raw_dataset=train_dataset.raw_eng, tokenizer=en_tokenizer)

#### Convert To Tensor

In [8]:
# 사전 데이터를 기반으로 데이터셋을 텐서로 변환
to_tensor = (
    transforms.to_tensor(ko_vocab, tokenizer=ko_tokenizer),
    transforms.to_tensor(en_vocab, tokenizer=en_tokenizer)
)

train_dataset.transform(transform=to_tensor)
valid_dataset.transform(transform=to_tensor)
test_dataset.transform(transform=to_tensor)

Using Special Tokens - PAD_IDX: 0, UNK_IDX: 1
Using Special Tokens - PAD_IDX: 0, UNK_IDX: 1


In [9]:
# 데이터셋 형태 확인
sample = list(zip(*train_dataset[0:5]))+list(zip(*train_dataset[500:505]))
for i, (kor, eng) in enumerate(sample):
    print(i, kor, eng)

0 tensor([196,   4,   0,   0,   0,   0,   0,   0,   0,   0]) tensor([452,   4,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])
1 tensor([519,   4,   0,   0,   0,   0,   0,   0,   0,   0]) tensor([1902,    4,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])
2 tensor([761,  19,   0,   0,   0,   0,   0,   0,   0,   0]) tensor([1343,   44,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])
3 tensor([761,   4,   0,   0,   0,   0,   0,   0,   0,   0]) tensor([1343,    4,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])
4 tensor([380,   5,   0,   0,   0,   0,   0,   0,   0,   0]) tensor([131,   8,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])
5 tensor([ 780, 1579,    4,    0,    0,    0,    0,    0,    0,    0]) tensor([1986,  879,    4,    0,    0,    0,    0,    0,    0,    0,    0,    0])
6 tensor([  12, 2864,    5,    0,    0,    0,    0,    0,    0,    0]) tensor([754,   5, 246,   8,   0,   0,   0,   0,   0,   0,   0,   0])
7 tensor([ 23, 793,   

#### Data Loader

In [10]:
# 배치 크기 결정 후 데이터 로더 생성
batch_size = 128

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataload = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset)//20)

## Model Definition
> RNN/GRU 레이어를 하나만 사용하였던 지난 주차와는 달리, 레이어를 여러 층으로 쌓는 방식의 Encoder와 Decoder 모델을 사용

> 인코더는 한국어를 해석하고, 디코더는 영어를 생성하는 방식으로 역할을 나눠서 번역을 수행


> 참고사항:
>> 아래 예시 모델은 공부하기 쉬운 단순한 모델로 번역에 있어 매우 뛰어난 성능을 보이는 모델은 아닙니다.
>> 최신 기술 트렌드는 Transformers를 사용하는 것입니다.
>> 혹시 관심이 있다면 [Transformer 레이어](https://pytorch.org/docs/stable/nn.html#transformer-layers)를 사용하는 코드로 변경해서 진행해보기 바랍니다.

In [11]:
import torch.nn as nn

In [12]:
input_vocab = ko_vocab.get_stoi()
word_2_idx = en_vocab.get_stoi()
idx_2_word = en_vocab.get_itos()

In [13]:
len(input_vocab), len(word_2_idx), len(idx_2_word)

(7971, 3474, 3474)

In [14]:
class Encoder(nn.Module):
    EMBEDDING_SIZE = 128

    def __init__(
            self,
            model:nn.Module,
            height:int,
            hidden:int,
            dropout:float = 0.2
    ):
        self.input_vocab = globals()['input_vocab']
        super(Encoder,self).__init__()
        self.height = height
        self.hidden = hidden
        self.embedding = nn.Embedding(len(self.input_vocab), self.EMBEDDING_SIZE)
        self.model = model(
            self.EMBEDDING_SIZE,
            hidden,
            num_layers=height,
            batch_first = True,
            dropout=dropout
        )

    def forward(self,x):
        x = self.embedding(x)
        x = F.relu(x)
        _,x = self.model(x)
        return x

In [15]:
BOS = vocabulary.Token.BOS
EOS = vocabulary.Token.EOS
PAD = vocabulary.Token.PAD

class Decoder(nn.Module):
    EMBEDDING_SIZE = 128
    def __init__(
            self,
            model:nn.Module,
            height:int,
            hidden:int,
            dropout:float = 0.2
    ):
        self.idx_2_word = globals()['idx_2_word']
        self.word_2_idx = globals()['word_2_idx']
        super(Decoder,self).__init__()
        self.height = height
        self.hidden = hidden
        self.embedding = nn.Embedding(len(self.idx_2_word), self.EMBEDDING_SIZE)
        self.model = model(
            self.EMBEDDING_SIZE,
            hidden,
            num_layers=height,
            batch_first = True,
            dropout=dropout
        )
        vocab_size = len(self.idx_2_word)
        # self.fc1 = nn.Linear(hidden,vocab_size//2)
        # self.fc2 = nn.Linear(vocab_size//2,vocab_size)
        self.fc1 = nn.Linear(hidden,hidden//2)
        self.fc2 = nn.Linear(hidden//2,vocab_size)

    def forward(self,x,cv=None):
        x = self.embedding(x)
        x = F.relu(x)
        output,last_hidden = self.model(x,cv)
        x = F.relu(self.fc1(output))
        x = self.fc2(x)
        return x,last_hidden

    def generate(self,cv):
        word = BOS
        cnt = 0
        while word != EOS:
            if cnt > 10:
                break
            x = torch.tensor(self.word_2_idx[word]).unsqueeze(0).unsqueeze(0).to('cuda')
            x,cv = self(x,cv)
            _,x = torch.max(x.view(-1,len(self.idx_2_word)),dim=1)
            word = self.idx_2_word[x.item()]
            print(word,end=" ")
            cnt += 1

In [16]:
height = 2
hidden = 64
encoder = Encoder(nn.GRU, height=height, hidden=hidden, dropout=0.3)
decoder = Decoder(nn.GRU, height=height, hidden=hidden, dropout=0.3)

In [25]:
def train_loop(encoder,decoder,dataset,epochs,lr,encoder_optimizer,decoder_optimizer,criterion):
    encoder.train()
    decoder.train()
    decoder.to('cuda')
    encoder.to('cuda')
    for i,epoch in enumerate(range(epochs)):
        running_loss = 0.0
        
        for step,batch in enumerate(dataset):
            srcs, tgts = batch
            encoder_input = torch.tensor([[input_vocab[token] for token in src] for src in srcs]).to('cuda')
            decoder_input = torch.tensor([[word_2_idx[token] for token in [BOS] + tgt] for tgt in tgts]).to('cuda')
            decoder_label = torch.tensor([[word_2_idx[token] for token in tgt + [EOS]] for tgt in tgts]).to('cuda')

            context_vector = encoder(encoder_input)
            x,_ = decoder(decoder_input,context_vector)

            decoder_label = torch.nn.functional.one_hot(decoder_label, num_classes=11).float()
            loss = criterion(x,decoder_label)
            running_loss += loss.item()
            loss.backward()
            encoder_optimizer.step()
            encoder_optimizer.zero_grad()
            decoder_optimizer.step()
            decoder_optimizer.zero_grad()
        print(f"epoch:{i+1}/{epochs} loss: {running_loss}")

참고 : 언어 번역의 성능 점수를 기록하려면, ``nn.CrossEntropyLoss`` 함수가 단순한
패딩을 추가하는 부분을 무시할 수 있도록 해당 색인들을 알려줘야 합니다.



In [26]:
lr = 1e-4
encoder_optimizer = torch.optim.Adam(encoder.parameters(),lr)
decoder_optimizer = torch.optim.Adam(decoder.parameters(),lr)

PAD_IDX = ko_vocab[PAD]
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [27]:
train_loop(encoder,decoder,train_dataset,100,lr,encoder_optimizer,decoder_optimizer,criterion)

TypeError: iteration over a 0-d tensor

마지막으로 이 모델을 훈련하고 평가합니다 :



In [ ]:
import math
import time


def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

## 다음 단계

- ``torchtext`` 를 사용한 Ben Trevett의 튜토리얼을 [이곳](https://github.com/bentrevett/)_ 에서 확인할 수 있습니다.
- ``nn.Transformer`` 와 ``torchtext`` 의 다른 기능들을 이용한 다음 단어 예측을 통한 언어 모델링 튜토리얼을 살펴보세요.

